In [1]:
from pyomo.environ import (
    ConcreteModel,
    Objective,
    Expression,
    value,
    Var,
    Param,
    Constraint,
    Set,
    Var,
    Block,
    SolverFactory,
    TransformationFactory,
    assert_optimal_termination,
    check_optimal_termination,
    log,
    log10,
    units as pyunits,
)

from pyomo.network import Port
from idaes.core import FlowsheetBlock
from idaes.core.solvers.get_solver import get_solver

from idaes.core.util.model_statistics import *
from idaes.core.util.scaling import *

from idaes.core import FlowsheetBlock, UnitModelCostingBlock

from watertap.core.zero_order_costing import ZeroOrderCosting, _get_tech_parameters
from watertap.core.util.infeasible import *
from watertap.costing import WaterTAPCosting

import json
from os.path import join, dirname
from math import floor, ceil

import pytest

from io import StringIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from idaes.core.util.model_statistics import (
    degrees_of_freedom,
    number_variables,
    number_total_constraints,
    number_unused_variables,
)

from IPython.display import clear_output
from copy import deepcopy
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.property_models.water_prop_pack import WaterParameterBlock
from watertap_contrib.seto.costing import  SETOWaterTAPCosting #SETOZeroOrderCosting,

from watertap_contrib.seto.costing import (
    TreatmentCosting,
    EnergyCosting,
    SETOSystemCosting,
)

# from watertap_contrib.seto.solar_models.zero_order import PhotovoltaicZO
# from watertap_contrib.seto.energy import solar_energy
# from watertap_contrib.seto.core import SETODatabase, PySAMWaterTAP
from watertap_contrib.seto.unit_models.chemical_softening_0D import ChemicalSoftening0D

from watertap_contrib.seto.property_models.basic_water_properties import (
    BasicWaterParameterBlock,
)

solver = get_solver()

def get_ion_config(ions):
    #neutral_solutes = ["TSS", "TDS", "TOC", "NH3", "SiO2"]
    if not isinstance(ions, list):
        ions = [ions]
    #ions = ions + neutral_solutes

    mw_data = {
        "Na_+": 23e-3,
        "Ca_2+": 40e-3,
        "Cl_-": 35e-3,
        "Mg_2+": 24e-3,
        "SO4_2-": 96e-3,
        "NH3": 17.03e-3,
        "SiO2": 60.08e-3,
        "HCO3_-": 61.02e-3,
        "Alkalinity_2-": 31.01736e-3,
        "CO3_2-": 60.01e-3,
        "CO2": 44e-3,
    }
    charge_data = {
        "Na_+": 1,
        "Ca_2+": 2,
        "Cl_-": -1,
        "Mg_2+": 2,
        "SO4_2-": -2,
        "HCO3_-": -1,
        "CO3_2-": -2,
        "Alkalinity_2-": -2,
    }
    ion_config = {
        "solute_list": [],
        "mw_data": {"H2O": 18e-3},
        "mw_data": {},
        "charge": {},
    }

    for ion in ions:
        ion_config["solute_list"].append(ion)

        if ion in charge_data.keys():
            ion_config["charge"][ion] = charge_data[ion]
        if ion in mw_data.keys():
            ion_config["mw_data"][ion] = mw_data[ion]
    return ion_config


In [2]:
component_list = ["Ca_2+","Mg_2+","Alkalinity_2-","TSS"]
input_config_dict = get_ion_config(component_list)
input_config_dict

{'solute_list': ['Ca_2+', 'Mg_2+', 'Alkalinity_2-', 'TSS'],
 'mw_data': {'Ca_2+': 0.04, 'Mg_2+': 0.024, 'Alkalinity_2-': 0.03101736},
 'charge': {'Ca_2+': 2, 'Mg_2+': 2, 'Alkalinity_2-': -2}}

In [3]:
ca_in = 0.075  * pyunits.kg / pyunits.m**3  # g/L = kg/m3
mg_in = 0.0061 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
sio2_in = 0.054 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
alk_in = 0.196 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
TSS_in = 0.20 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
CO2_in  = 0.072*1.612 * pyunits.kg / pyunits.m**3 
q_in = 50000 * pyunits.m**3 / pyunits.day  # m3/d
rho = 1000 * pyunits.kg / pyunits.m**3

pH_in = 7
temp_in = 293.15  # K

ca_mass_flow = pyunits.convert((ca_in * q_in), to_units=pyunits.kg / pyunits.s)()
mg_mass_flow = pyunits.convert((mg_in * q_in), to_units=pyunits.kg / pyunits.s)()
sio2_mass_flow = pyunits.convert((sio2_in * q_in), to_units=pyunits.kg / pyunits.s)()
h2o_mass_flow = pyunits.convert((rho * q_in), to_units=pyunits.kg / pyunits.s)()



In [37]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
# m.fs.properties = ChemSofteningParameterBlock(**input_config_dict)
m.fs.properties =  BasicWaterParameterBlock(solute_list=component_list)

m.fs.soft = soft = ChemicalSoftening0D(
    property_package=m.fs.properties, silica_removal= False ,softening_procedure_type= 'single_stage_lime'
)
m.fs.properties.calculate_scaling_factors()
prop_in = soft.properties_in[0]
# prop_out = soft.properties_out[0]
# prop_waste = soft.properties_waste[0]

print(f"DOF = {degrees_of_freedom(m)}")


DOF = 17


In [31]:
# prop_in.flow_mass_comp["Ca_2+"].fix(ca_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp["Ca_2+"], 100)
# prop_in.flow_mass_comp[ "Mg_2+"].fix(mg_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "Mg_2+"], 100)
# prop_in.flow_mass_comp[ "SiO2"].fix(sio2_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "SiO2"], 100)
# prop_in.flow_mass_comp["H2O"].fix(h2o_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "H2O"], 1e-2)

# prop_out.flow_vol.fix(0.1*q_in)
# prop_waste.flow_vol.fix(0.9*q_in)

In [40]:
prop_in.flow_vol.fix(q_in)

prop_in.conc_mass_comp["Ca_2+"].fix(ca_in)
prop_in.conc_mass_comp["Mg_2+"].fix(mg_in)
# prop_in.conc_mass_comp["SiO2"].fix(sio2_in)
prop_in.conc_mass_comp["Alkalinity_2-"].fix(alk_in)
prop_in.conc_mass_comp["TSS"].fix(TSS_in)

set_scaling_factor(prop_in.flow_vol,100)
set_scaling_factor(prop_in.conc_mass_comp["Ca_2+"],100)
set_scaling_factor(prop_in.conc_mass_comp["Mg_2+"],100)
# set_scaling_factor(prop_in.conc_mass_comp["SiO2"],100)
set_scaling_factor(prop_in.conc_mass_comp["Alkalinity_2-"],100)
set_scaling_factor(prop_in.conc_mass_comp["TSS"],100)

soft.ca_eff_target.fix()
soft.mg_eff_target.fix()

soft.no_of_mixer.fix(1)
soft.no_of_floc.fix(2)
soft.retention_time_mixer.fix(0.4)
soft.retention_time_floc.fix(25)
soft.retention_time_sed.fix(130)
soft.retention_time_recarb.fix(20)
soft.frac_vol_recovery.fix()
soft.removal_efficiency.fix()
soft.CO2_CaCO3.fix(CO2_in)
soft.vel_gradient_mix.fix(300)
soft.vel_gradient_floc.fix(50)
# soft.excess_CaOH.fix(0)
# m.fs.soft.MgCl2_dosing.fix()
soft.initialize()

print(f"DOF = {degrees_of_freedom(m)}")

2023-03-20 15:42:55 [INFO] idaes.init.fs.soft: Initialization Step 1a Complete.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft.properties_out: State Released.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft: Initialization Step 1b Complete.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft.properties_waste: State Released.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft: Initialization Step 1c Complete.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft: Initialization Step 2 optimal - Optimal Solution Found.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft.properties_in: State Released.
2023-03-20 15:42:56 [INFO] idaes.init.fs.soft: Initialization Complete: optimal - Optimal Solution Found
DOF = 0


In [41]:
calculate_scaling_factors(m)
list(unscaled_variables_generator(m))

2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_out[0.0].conc_mass_comp[Ca_2+]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_out[0.0].conc_mass_comp[Mg_2+]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_out[0.0].conc_mass_comp[Alkalinity_2-]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_out[0.0].conc_mass_comp[TSS]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_waste[0.0].conc_mass_comp[Ca_2+]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_waste[0.0].conc_mass_comp[Mg_2+]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.soft.properties_waste[0.0].conc_mass_comp[Alkalinity_2-]
2023-03-20 15:42:58 [WARNING] idaes.core.util.scaling: M

In [22]:
number_unused_variables(m)

5

In [9]:
print_infeasible_constraints(m)

In [10]:
print(prop_in.conc_mass_comp["Mg_2+"].value)
print(prop_out.conc_mass_comp["Mg_2+"].value)
print(prop_waste.conc_mass_comp["Mg_2+"].value)

0.0061
0.004854368932038835
49.80941747572762


In [36]:
soft.sludge_prod.value

0.36094398148148144

In [6]:
m.fs.costing = TreatmentCosting()
soft.costing = UnitModelCostingBlock(flowsheet_costing_block = m.fs.costing)
m.fs.costing.cost_process()
m.fs.costing.add_LCOW(prop_in.flow_vol)


2023-03-20 14:24:12 [WARNING] idaes.core.base.costing_base: fs.soft capital_cost component has a lower bound less than zero. Be aware that this may result in negative costs during optimization.
2023-03-20 14:24:12 [WARNING] idaes.core.base.costing_base: fs.soft fixed_operating_cost component has a lower bound less than zero. Be aware that this may result in negative costs during optimization.


In [13]:
print(f"DOF = {degrees_of_freedom(m)}")

DOF = 0


In [7]:
results = solver.solve(m)

print(f"DOF = {degrees_of_freedom(m)}")
print(results.solver.termination_condition.swapcase())
print_infeasible_constraints(m)


DOF = 0
OPTIMAL


In [8]:
m.fs.soft.Mg_hardness_nonCaCO3()

0.01663199999999998

In [37]:
print(pyunits.get_units(prop_in.flow_vol))

m**3/s


In [8]:
print(m.fs.soft.volume_mixer.value)
print(m.fs.soft.volume_floc.value)
print(m.fs.soft.volume_sed.value)
print(m.fs.soft.volume_recarb.value)
print(m.fs.soft.CO2_first_basin.value)

13.888888888888886
1736.1111111111106
4513.888888888888
694.4444444444443
847.0


In [13]:
(m.fs.soft.sludge_prod.value)

0.36094398148148144

In [9]:
print(m.fs.soft.costing.mix_tank_capital_cost.value)
print(m.fs.soft.costing.floc_tank_capital_cost.value)
print(m.fs.soft.costing.sed_basin_capital_cost.value)
print(m.fs.soft.costing.recarb_basin_capital_cost.value)
print(m.fs.soft.costing.recarb_basin_source_capital_cost.value)
print(m.fs.soft.costing.lime_feed_system_capital_cost.value)
print(m.fs.soft.costing.admin_capital_cost.value)
# m.fs.soft.volume_floc.value

39803.320656448836
526325.0016145604
1062683.6511610087
203901.92518795643
206831.68693986651
208934.44577817747
287839.27983806236


In [10]:
print(m.fs.soft.costing.mix_tank_op_cost.value)
print(m.fs.soft.costing.floc_tank_op_cost.value)
print(m.fs.soft.costing.sed_basin_op_cost.value)
print(m.fs.soft.costing.recarb_basin_op_cost.value)
print(m.fs.soft.costing.lime_feed_op_cost.value)
print(m.fs.soft.costing.lime_sludge_mngt_op_cost.value)
print(m.fs.soft.costing.admin_op_cost.value)

24168.658988826148
24102.20176214742
19100.27097544179
20494.04621691925
421787.0232411073
439456.68954925315
289576.0903340209


In [16]:
print(m.fs.soft.costing.floc_power.value)

104.16666666666664


In [20]:
m.fs.objective = Objective(expr=m.fs.costing.LCOW)
results = solver.solve(m)

In [21]:
m.fs.costing.LCOW.value

0.8054329238702729

In [22]:
# Creating removal dictionary

removal_eff_dict = dict(
            zip([
                x for x in component_list if x not in ["Ca_2+","Mg_2+"]
                ]
                ,
                [   
                    0.7 if j != "TDS" else 1e-3
                    for j in component_list 
                ],
            )
        )

print(removal_eff_dict)

{'SiO2': 0.7, 'Alkalinity_2-': 0.7}


In [23]:

new_list = [x for x in component_list if x not in ["Ca_2+","Mg_2+"]]
print(new_list)

['SiO2', 'Alkalinity_2-']
